# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create is assigned to rnmehta5@gmail.com and will expire on April 20, 2017. For commercial licensing options, visit https://dato.com/buy/.


2016-05-27 15:16:03,684 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: /tmp/graphlab_server_1464387362.log


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.884781     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.954325     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [6]:
def num_zero_coeffs(model):
    coef_dict = {}
    num_zeros = 0
    for coeff in model.get('coefficients'):
        if coeff['value'] != 0.0:
            coef_dict[coeff['name']] = coeff['value']
    return coef_dict
print num_zero_coeffs(model_all)

{'bathrooms': 8468.531086910072, 'sqft_above': 20.024722417091304, 'sqft_living_sqrt': 350.0605533860648, 'grade': 842.0680348976282, 'sqft_living': 24.420720982445214, '(intercept)': 274873.0559504957}


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [7]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [9]:
l1_penalty = np.logspace(1, 7, num=13)
models = []
rss_validate = []
rss_test = []
for i in range(0, len(l1_penalty)):
    models.append(graphlab.linear_regression.create(training, target= 'price', features= all_features, 
                                                      validation_set= None, l1_penalty= l1_penalty[i], 
                                                      l2_penalty= 0, verbose = False))
for i in range(0, len(l1_penalty)):
    rss_validate.append((validation['price'] - models[i].predict(validation)).apply(lambda x: x**2).sum())

In [11]:
for i in range(0, len(rss_validate)):
    print str(l1_penalty[i]) + "    " + str(rss_validate[i])

10.0    6.25766285142e+14
31.6227766017    6.25766285362e+14
100.0    6.25766286058e+14
316.227766017    6.25766288257e+14
1000.0    6.25766295212e+14
3162.27766017    6.25766317206e+14
10000.0    6.25766386761e+14
31622.7766017    6.25766606749e+14
100000.0    6.25767302792e+14
316227.766017    6.25769507644e+14
1000000.0    6.25776517727e+14
3162277.66017    6.25799062845e+14
10000000.0    6.25883719085e+14


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [20]:
test_rss = []
for i in range(0,len(model)):
    test_rss.append((model[i].predict(testing) - testing['price']).apply(lambda x: x**2).sum())
for i in range(0,len(test_rss)):
    print str(l1_penalties[i]) + "     " + str(test_rss[i])

10.0     1.56983602382e+14
31.6227766017     1.56983602351e+14
100.0     1.56983602254e+14
316.227766017     1.56983601949e+14
1000.0     1.56983600982e+14
3162.27766017     1.56983597925e+14
10000.0     1.5698358826e+14
31622.7766017     1.56983557704e+14
100000.0     1.56983461152e+14
316227.766017     1.56983156601e+14
1000000.0     1.56982201232e+14
3162277.66017     1.56979257164e+14
10000000.0     1.56972779669e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [16]:
print len(num_zero_coeffs(models[0]).keys())

18


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [17]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [18]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [22]:
try_models = []
num_non_zeroes = []
l1_penalty_min = l1_penalty_values[0]
l1_penalty_max = l1_penalty_values[len(l1_penalty_values)-1]
for i in range(0,len(l1_penalty_values)):
    try_models.append(graphlab.linear_regression.create(training, target= 'price', features= all_features, 
                                                        validation_set= None, l1_penalty= l1_penalty_values[i], 
                                                        l2_penalty= 0, verbose = False))
    num_non_zeroes.append(try_models[i]['coefficients']['value'].nnz())
    if num_non_zeroes[i] > max_nonzeros and l1_penalty_values[i] > l1_penalty_min:
        l1_penalty_min = l1_penalty_values[i]
    if num_non_zeroes[i] < max_nonzeros and l1_penalty_values[i] < l1_penalty_max:
        l1_penalty_max = l1_penalty_values[i]
print num_non_zeroes

[18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 15, 15, 13, 12, 10, 6, 5, 3, 1, 1]


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [23]:
print l1_penalty_min 
print l1_penalty_max

2976351441.63
3792690190.73


***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

In [24]:
print l1_penalty_values

[  1.00000000e+08   1.27427499e+08   1.62377674e+08   2.06913808e+08
   2.63665090e+08   3.35981829e+08   4.28133240e+08   5.45559478e+08
   6.95192796e+08   8.85866790e+08   1.12883789e+09   1.43844989e+09
   1.83298071e+09   2.33572147e+09   2.97635144e+09   3.79269019e+09
   4.83293024e+09   6.15848211e+09   7.84759970e+09   1.00000000e+10]


## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [25]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [27]:
line_space_models = []
rss_line_space_val = []
for i in range(0,len(l1_penalty_values)):
    line_space_models.append(graphlab.linear_regression.create(training, target= 'price', features= all_features, 
                                                               validation_set= None, l1_penalty= l1_penalty_values[i], 
                                                               l2_penalty= 0, verbose = False))
    rss_line_space_val.append((validation['price'] - line_space_models[i].predict(validation)).apply(lambda x: x**2).sum())
for i in range(0,len(l1_penalty_values)):
    print str(l1_penalty_values[i]) + "    " + str(rss_line_space_val[i])

2976351441.63    9.66925692362e+14
3019316638.95    9.74019450085e+14
3062281836.27    9.81188367942e+14
3105247033.59    9.89328342459e+14
3148212230.92    9.98783211266e+14
3191177428.24    1.00847716702e+15
3234142625.56    1.01829878055e+15
3277107822.88    1.02824799221e+15
3320073020.2    1.03461690923e+15
3363038217.52    1.03855473594e+15
3406003414.84    1.04323723787e+15
3448968612.16    1.04693748875e+15
3491933809.48    1.05114762561e+15
3534899006.81    1.05599273534e+15
3577864204.13    1.06079953176e+15
3620829401.45    1.0657076895e+15
3663794598.77    1.06946433543e+15
3706759796.09    1.07350454959e+15
3749724993.41    1.07763277558e+15
3792690190.73    1.08186759232e+15


In [31]:
for i in range(0,len(line_space_models)):
    print str(i) + "    " + str(l1_penalty_values[i]) + "    " + str(line_space_models[i]['coefficients']['value'].nnz())

0    2976351441.63    10
1    3019316638.95    10
2    3062281836.27    10
3    3105247033.59    10
4    3148212230.92    10
5    3191177428.24    10
6    3234142625.56    10
7    3277107822.88    10
8    3320073020.2    8
9    3363038217.52    8
10    3406003414.84    8
11    3448968612.16    7
12    3491933809.48    7
13    3534899006.81    7
14    3577864204.13    7
15    3620829401.45    6
16    3663794598.77    6
17    3706759796.09    6
18    3749724993.41    6
19    3792690190.73    6


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [34]:
for i in range(0, len(line_space_models[11]['coefficients']['name'])):
    print line_space_models[11]['coefficients']['name'][i] + "    " + str(line_space_models[11]['coefficients']['value'][i])

(intercept)    222253.192544
bedrooms    661.722717782
bedrooms_square    0.0
bathrooms    15873.9572593
sqft_living    32.4102214513
sqft_living_sqrt    690.114773313
sqft_lot    0.0
sqft_lot_sqrt    0.0
floors    0.0
floors_square    0.0
waterfront    0.0
view    0.0
condition    0.0
grade    2899.42026975
sqft_above    30.0115753022
sqft_basement    0.0
yr_built    0.0
yr_renovated    0.0
